# Import all the libraries here

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def snake_columns(data):
    """
    Standarize and returns snake_case columns
    """
    data.columns = [column.lower().replace(' ', '_') for column in data.columns]

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# Load the data here

In [ ]:
data = pd.read_csv("../dataset/training_data.csv", sep='\t', names=["label", "text"])
df = data.copy()

In [ ]:
df.shape

In [ ]:
df.head(10)

# Exploratory Data Analysis

In [ ]:
# null values
df.isnull().sum()

In [ ]:
# distribution of label
sns.countplot(x=data["label"])
plt.show()

In [ ]:
data["label"].value_counts()

**Overview**
- There is no null value in the dataset.
- `label` has two values: 0 (fake news) & 1 (real news)
- `label` has almost equal values of 0 and 1. We might need to remove some 0 values to unbias the output.
- Number of rows: 34152 

In [ ]:
# sending target to the last
target = data.pop("label")
data["label"] = target

In [ ]:
data

**Comment**
- The whole dataset is my training data. So I don't need to do train_test_split.

# Feature Transformation

In [ ]:
train_data = data

In [ ]:
# cleaning string characters
train_data.iloc[:, 0].replace("[^a-zA-Z]", " ", regex=True, inplace=True)

In [ ]:
train_data["text"]

In [ ]:
train_data["text"] = train_data["text"].astype(str)

In [ ]:
train_data.info()

In [ ]:
train_headlines = []
for row in range(0, len(train_data.index)):
    train_headlines.append(train_data.iloc[row, 0])

In [ ]:
train_headlines

# Count Vectorizer

In [ ]:
countvector = CountVectorizer(ngram_range=(2, 2))

In [ ]:
traindataset = countvector.fit_transform(train_headlines)

In [ ]:
traindataset

# Model: Random Forest

In [ ]:
train_data.head(3)

In [ ]:
traindataset[:5]

In [ ]:
traindataset[:5].toarray()

In [ ]:
ground_truth = train_data["label"]

In [ ]:
# Implement RandomForest Classifier with additional parameters
randomclassifier = RandomForestClassifier(n_estimators=200, 
criterion='entropy', max_depth=10, random_state=42)

# n_estimators: number of trees in the forest
# criterion: function to measure the quality of a split. Entropy is the measure of impurity
# max_depth: maximum depth of the tree
# random_state: seed for random number generator

# Model training

In [ ]:
randomclassifier.fit(traindataset, ground_truth)

# Cross validation

In [ ]:
# Evaluate the model using cross-validation
scores = cross_val_score(randomclassifier, traindataset, ground_truth, cv=5)

In [ ]:
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean()}")

# Predictions

In [ ]:
# load the testing data
test_data = pd.read_csv("../dataset/testing_data.csv", sep='\t', names=["label", "text"])


In [ ]:
test_data.shape

In [ ]:
test_data.head(2)

In [ ]:
test_data["label"].value_counts()

In [ ]:
test_headlines = []
for row in range(0, len(test_data.index)):
    test_headlines.append(test_data.iloc[row, 1])

In [ ]:
test_headlines

In [ ]:
testdataset = countvector.transform(test_headlines)

In [ ]:
# Make predictions on the test data
predictions = randomclassifier.predict(testdataset)

In [ ]:
predictions

In [ ]:
sns.countplot(x=predictions)
plt.show()

In [ ]:
print(f"Size of ground truth: {len(test_data["label"])}")
print(f"Size of predictions: {len(predictions)}")

In [ ]:
test_data["predictions"] = predictions
test_data

# Model: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr = LogisticRegression()

In [ ]:
traindataset

In [ ]:
model_lr.fit(traindataset, ground_truth)

In [ ]:
# Evaluate the model using cross-validation
scores_lr = cross_val_score(model_lr, traindataset, ground_truth, cv=5)

In [ ]:
print(f"Cross-validation scores: {scores_lr}")
print(f"Mean cross-validation score: {scores_lr.mean()}")

In [ ]:
# Make predictions on the test data
predictions_lr = model_lr.predict(testdataset)

In [ ]:
sns.countplot(x=predictions_lr)
plt.show()

In [ ]:
test_data

In [ ]:
test_data["predictions_log_reg"] = predictions_lr

In [ ]:
test_data.rename({"predictions" : "predictions_rand_for"}, inplace=True)

In [ ]:
test_data

In [ ]:
# save the files
test_data.to_csv("../dataset/predictions.csv")

**Summary**
- `LogisticRegression` works better than `RandomForest`